In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn.metrics import confusion_matrix as cm

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier

starbucks = pd.read_csv("D:/UTC/SY09/Projet/tests locaux/clean_starbucks.csv")
starbucksX = starbucks.drop(columns=["product_name", "milk", "size", "whip", "serv_size_m_l", "Category"])
#starbucksX = starbucks.drop(columns=["product_name", "size", "Category"])
starbucksZ = starbucks.Category

In [10]:
def confusion_matrix(z_pred, z_test):
    z_comp = pd.DataFrame({"Prediction": z_pred, "Realite": z_test})
    categories = ["coffee", "tea", "frappuccino", "chocolate", "other"]
    res = {}
    
    for i in categories : # attendu
        values = []
        for j in categories: # predit
            values.append(100 * len(z_comp[ (z_comp["Realite"]==i) & (z_comp["Prediction"]==j)]) / len(z_comp[z_comp["Realite"]==i]))
        res[i] = values
        
    conf = pd.DataFrame(res)
    indexes = []
    for i in conf.columns:
        indexes.append(i + "_pred")
    conf["index"] = indexes
    return conf

In [51]:
def big_confusion_matrix(cls, nb_models) :

    matrices = []

    # On stocke les matrices de confusion de chaque modèle généré
    
    for i in range(0,nb_models):

        X_train, X_test, z_train, z_test = train_test_split(starbucksX, starbucksZ, test_size=0.33, random_state=i)
        cls.fit(X_train, z_train)
        z_pred = cls.predict(X_test)

        conf = confusion_matrix(z_pred, z_test)
        #print(conf)
        #print(conf.melt(id_vars=["index"], value_name=i))
        matrices.append(conf.melt(id_vars=["index"], value_name=i))

    # On construit une matrice qui stocke les pourcentages obtenus dans des colonnes nommées de 0 à nb_models  
    
    CF = matrices[0]

    for i in range(1,len(matrices)):
        CF = pd.concat([CF, matrices[i][i]], axis = 1)

    # On calcule les moyennes et écarts-types de chaque case de la matrice de confusion
    
    CF["average"] = CF[range(0, nb_models)].mean(axis=1).round(decimals=1).astype(str)
    CF["std"] = CF[range(0, nb_models)].std(axis=1).round(decimals=1).astype(str)
    
    # On met en forme le résultat puis on pivote la table pour bien avoir la matrice de confusion attendue
    
    CF["value"] = CF["average"] + " ± " + CF["std"]
    CF = CF.drop(columns=range(0, nb_models))
    CF = CF.drop(columns=["average","std"])
    CF = CF.pivot(index='index', columns='variable')['value']
    
    return CF
    
nb_models = 100

lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
lr = LogisticRegression(max_iter=5000)

print(big_confusion_matrix(lda, nb_models))
print(big_confusion_matrix(qda, nb_models))
print(big_confusion_matrix(lr, nb_models))

variable           chocolate      coffee frappuccino        other         tea
index                                                                        
chocolate_pred    71.3 ± 8.1   0.1 ± 0.2   0.6 ± 0.6    0.0 ± 0.0   1.0 ± 1.4
coffee_pred        0.1 ± 0.6  87.4 ± 2.6   0.9 ± 0.8    0.0 ± 0.0  10.4 ± 3.4
frappuccino_pred   0.0 ± 0.0   6.3 ± 1.6  97.4 ± 1.4   10.0 ± 8.6   0.0 ± 0.0
other_pred         5.5 ± 3.7   0.0 ± 0.0   0.0 ± 0.0  52.3 ± 15.2   2.0 ± 2.0
tea_pred          23.1 ± 7.6   6.3 ± 1.9   1.0 ± 0.9  37.7 ± 14.0  86.6 ± 3.8
variable           chocolate      coffee frappuccino        other         tea
index                                                                        
chocolate_pred    95.0 ± 4.0   0.0 ± 0.0   0.0 ± 0.0    0.0 ± 0.0   0.1 ± 0.4
coffee_pred        4.4 ± 3.8  84.8 ± 2.6   1.1 ± 0.9    7.4 ± 9.9   8.3 ± 3.9
frappuccino_pred   0.0 ± 0.0   6.1 ± 1.6  98.7 ± 1.0   8.7 ± 10.2   0.1 ± 0.3
other_pred         0.6 ± 1.5   1.5 ± 1.1   0.2 ± 0.4  77.0 ± 17.

C:\Users\leond\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\leond\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

variable           chocolate      coffee frappuccino        other         tea
index                                                                        
chocolate_pred    97.1 ± 3.7   0.0 ± 0.1   0.3 ± 0.4    5.1 ± 5.9   0.0 ± 0.0
coffee_pred        0.0 ± 0.0  89.5 ± 2.1   6.5 ± 2.0    0.0 ± 0.0  11.2 ± 3.6
frappuccino_pred   0.0 ± 0.0   6.1 ± 1.6  92.3 ± 2.3    7.1 ± 9.6   0.0 ± 0.0
other_pred         1.4 ± 2.6   0.0 ± 0.1   0.0 ± 0.0  61.9 ± 15.6   1.2 ± 1.2
tea_pred           1.6 ± 2.6   4.4 ± 1.6   1.0 ± 1.0  25.9 ± 12.9  87.6 ± 3.8


Démarche : on produit 100 modèles de chaque et calcule la moyenne et l'écart-type de chaque case des matrices de confusion. Pour des raisons techniques, on n'utilise pas les modèles issus de la cross-validation


LDA : 
- Chocolats : thés, (autres)
- Cafés : (frappuccinos, thés)
- Frappuccinos : peu confondus
- Autres : thés, frappuccinos
- Thés : cafés

QDA : 
- Chocolats : (cafés)
- Cafés : (frappuccinos, thés)
- Frappuccinos : peu confondus
- Autres : un peu plus précis que LDA, thés, frappuccino
- Thés : moins précis sur les thés, cafés et autres

==> À ce stade, on privilégie les modèles LDA qui sont plus précis sur les thés. On perd en précision sur la catégorie autres ce qui est moins grave.

LR : 
- Chocolats : peu confondus (+26)
- Cafés : (frappuccinos, thés), taux un peu meilleurs que LDA (+2)
- Frappuccinos : (cafés), moins bon de LDA (-5)
- Autres : thés, (frappuccino, chocolat), moins bon que LDA
- Thés : cafés, proche de LDA

==> On privilégie LR, qui prédit beaucoup mieux les chocolats, ce qui nous coute une faible baisse de précision pour les frappuccinos. Ces modèles ont de plus hauts pourcentages de bonnes prédictions dans toutes les classes.